In [4]:
!pip install python-dotenv
!pip install ipykernel

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
#Langsmith Tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

##Data Ingestion -- From the website we need to scrap the data
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
## Convert the data to chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
all_splits = text_splitter.split_documents(data)

In [7]:
## Embeded text to vectors
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
## Store data to FAISS db
from langchain.vectorstores import FAISS
db = FAISS.from_documents(all_splits, embeddings)

/var/folders/j4/682zxx4166d4gj9yjn9j6mc00000gn/T/ipykernel_5082/3902904122.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/bibhukalyanojha/tensorflow-test/env/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/bibhukalyanojha/tensorflow-test/env/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
## Query from vector stored DB
result=db.similarity_search("Subgoal and decomposition")
result[0].page_content

'Planning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory'

In [34]:
## Retrival chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
import streamlit as st
from langchain_core.output_parsers import StrOutputParser

##LLAMA3 model
llm=Ollama(model="LLAMA3.2")
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    """
)

## Document chain
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
#Prompt template
# prompt=ChatPromptTemplate.from_messages(
#     [
#         ("system","You are a helpful assistant. Please respond to the question asked."),
#         ("user","Question:{question}")
#     ]
# )
##output parser
#output_parser=StrOutputParser()
## Document chain
#document_chain = prompt|llm|output_parser

In [25]:
result=document_chain.invoke({"question":"Subgoal and decomposition"})
result

"I'd be happy to help clarify subgoals and decomposition.\n\n**What is Subgoal and Decomposition?**\n\nSubgoals and decomposition are two fundamental concepts in problem-solving, planning, and decision-making. They are used to break down a complex problem into smaller, more manageable tasks (subgoals) that can be achieved one by one.\n\n**Subgoal:**\nA subgoal is a specific, achievable goal that contributes to the overall objective of a task or project. It's a smaller, more concrete target that can be accomplished by focusing on a particular aspect of the larger problem. Subgoals are often used to create a roadmap for achieving a higher-level goal.\n\n**Decomposition:**\nDecomposition is the process of breaking down a complex problem into its constituent parts, identifying subgoals, and creating a plan to achieve each one. It involves dividing the original problem into smaller, more manageable chunks, making it easier to understand, analyze, and solve.\n\n**Benefits of Subgoal and Deco

In [35]:
## Input --> Retriver --> vectorstoredb
retriver=db.as_retriever()
from langchain.chains import create_retrieval_chain
retrival_chain=create_retrieval_chain(retriver, document_chain)
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(...)
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x3c16a61f0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    '))])
            | Ollama(model='LLAMA3.2')
            | StrOutputParser(), config={'run_name': 'stuff_documents_chain'})
  }), config={'run_name': 'retrieval_chain'})

In [36]:
## Get the response from LLM
resonse=retrival_chain.invoke({"input":"Subgoal and decomposition"})
resonse

{'input': 'Subgoal and decomposition',
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='Planning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mist

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [37]:
resonse["answer"]

'According to the provided context, human agents can decompose tasks into smaller subgoals by using one of three methods:\n\n1. LLM (Large Language Model) with simple prompting, such as "Steps for XYZ.\\n1.", or "What are the subgoals for achieving XYZ?".\n2. Using task-specific instructions, e.g., "Write a story outline." for writing a novel.\n3. With human input.'